In [1]:
import random
import pandas as pd
import numpy as np
import os
import cv2
import copy

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from torchvision import transforms as T
from torchsummary import summary

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import warnings
import util

warnings.filterwarnings(action='ignore')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [2]:
CFG = {
    'IMG_SIZE':380,
    'EPOCHS':4,
    'LEARNING_RATE':5e-5,
    'BATCH_SIZE':12,
    'SEED':41,
    'ALPHA': 0.5
}

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [3]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=5):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b4(pretrained=True)
        self.backbone.classifier[-2] = nn.Dropout(0.7)
        self.backbone.classifier[-1] = nn.Linear(1792, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        return x

In [4]:
df = pd.read_csv('train_pic.csv', index_col=0)
df_labeled = df[df.loc[:, 'weather'] != -1]

train_unlabeld = df[df.loc[:, 'weather'] == -1]
train_labeled = df_labeled[: int(len(df_labeled) * 0.8)]
valid = df_labeled[int(len(df_labeled) * 0.8):]

In [5]:
class CustomDataset(Dataset):
    def __init__(self, path_list, label_list, tfms):
        self.path_list = path_list
        self.label_list = label_list
        self.tfms = tfms
        
    def __getitem__(self, index):
        image = cv2.imread(self.path_list[index])
        if self.tfms is not None:
            image = self.tfms(image = image)['image']

        if self.label_list is not None:
            label = np.zeros(5)
            label[int(self.label_list[index][0])] = 1
            label[3 + int(self.label_list[index][1])] = 1
            label = np.float32(label)
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.path_list)

In [6]:
tfms_train = A.Compose([A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
                        A.HorizontalFlip(p=0.5),
                        A.ColorJitter(p=0.5),
                        A.Rotate((-30,30), p=0.5),
                        A.Normalize(mean=(0.3192, 0.3201, 0.3083), std=(0.2132, 0.2072, 0.2059)),
                        ToTensorV2()])

tfms_test = A.Compose([A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
                        A.Normalize(mean=(0.3192, 0.3201, 0.3083), std=(0.2132, 0.2072, 0.2059)),
                        ToTensorV2()])

train_dataset = util.CustomDataset(train_labeled['path'].values, train_labeled.loc[:, ['weather', 'timing']].values, tfms_train)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=4)

train_unlabeld_dataset = util.CustomDataset(train_unlabeld['path'].values, None, tfms_test)
train_loader_unlabeled = DataLoader(train_unlabeld_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=4)

valid_dataset = util.CustomDataset(valid['path'].values, valid.loc[:, ['weather', 'timing']].values, tfms_test)
valid_loader = DataLoader(valid_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=4)

In [7]:
def train(model, optimizer, epochs, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.BCELoss().to(device)
    
    best_val_loss = 0
    best_model = None
    
    for epoch in range(1, epochs+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.float().to(device)
            optimizer.zero_grad()
            
            output = model(imgs)
            output = torch.concat([F.softmax(output[:, :3]), F.softmax(output[:, 3:])], dim=1)
            loss = criterion(output, target=labels)
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
            if len(train_loss) >= 5:
                print("\rtrain_loss: {0}".format(round(np.mean(train_loss[-5:]), 5)), end="")
        _val_loss = validation(model, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}]')

        if scheduler is not None:
            scheduler.step(_val_loss)
            
        if best_val_loss < _val_loss:
            best_val_acc = _val_loss
            best_model = copy.deepcopy(model)

    return best_model, best_val_loss

def validation(model, val_loader, device):
    model.eval()
    val_loss = []
    criterion = nn.BCELoss().to(device)
    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            probs = model(imgs)
            
            loss = criterion(F.softmax(input=probs), target=labels)
            
            probs  = probs.cpu().detach().numpy()
            labels = labels.cpu().detach().numpy()
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
    print(_val_loss)
    return _val_loss

def psudo_label(model, train_unlabeld, device):
    model.to(device)
    model.eval()
    train_dataset = CustomDataset(train_unlabeld.loc[:, 'path'].values, None, tfms_test)
    train_loader = DataLoader(train_dataset, batch_size = 1, shuffle=False, num_workers=0)
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            output = model(imgs)
            output = output.to('cpu').numpy()
            pred = np.stack([np.argmax(output[:, :3], axis=1),
                             np.argmax(output[:, 3:], axis=1)], axis = 1)
            preds += list(pred)
    return np.array(preds, int)

def detect_outliers(model, train_labeled, threshold, device):
    model.eval()
    result = []
    train_labeled_updated = copy.deepcopy(train_labeled)
    train_labeled_outlier = copy.deepcopy(train_labeled)
    train_dataset = CustomDataset(train_labeled.loc[:, 'path'].values, train_labeled.loc[:, ['weather', 'timing']].values, tfms_test)
    train_loader = DataLoader(train_dataset, batch_size = 1, shuffle=False, num_workers=0)

    criterion = nn.BCELoss().to(device)
    with torch.no_grad():
        index = 0
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.float().to(device)
            output = model(imgs)
            output = torch.concat([F.softmax(output[:, :3]), F.softmax(output[:, 3:])], dim=1)
            loss = criterion(output, target=labels)
            result.append((loss.item(), index))
            index += 1

    result = sorted(result, reverse=True)
    for loss, index in result:
        if loss < max(result[len(result)//10][0], threshold):
            train_labeled_outlier = train_labeled_outlier.drop(index = train_labeled.index[index])
        else:
            train_labeled_updated = train_labeled_updated.drop(index = train_labeled.index[index])
    return train_labeled_updated, train_labeled_outlier

In [8]:
model = BaseModel()
model.to(device)
model.eval()

optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG['LEARNING_RATE'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1,threshold_mode='abs',min_lr=1e-8, verbose=True)
best_val_loss = 0
train_labeled_updated = copy.deepcopy(train_labeled)
train_unlabeld_updated = copy.deepcopy(train_unlabeld)
for epoch in range(1, CFG['EPOCHS'] + 1):
    #labeled data 학습
    print("training labeled data...")
    train_dataset = util.CustomDataset(train_labeled_updated['path'].values, train_labeled_updated.loc[:, ['weather', 'timing']].values, tfms_train)
    train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=4)

    infer_model, val_loss = train(model=model, optimizer=optimizer, epochs=1, train_loader=train_loader, val_loader=valid_loader, scheduler=scheduler, device=device)
    if best_val_loss < val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'model.pth')

    #labeled data 이상치 제거
    print("detecting outliers...")
    train_labeled_updated , train_labeled_outlier = detect_outliers(model, train_labeled, threshold = CFG['ALPHA'], device=device)
    print(len(train_labeled_outlier), 'data removed from', len(train_labeled))
    train_unlabeld_updated = pd.concat([train_labeled_outlier, train_unlabeld])

    #psudo labeling
    print("psudo labeling...")
    psudo_labeled = psudo_label(model, train_unlabeld_updated, device)

    # create concat loader
    unlabeled_dataset = util.CustomDataset(train_unlabeld_updated['path'].values, psudo_labeled, tfms_test)
    unlabeled_loader = DataLoader(unlabeled_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=4)

    #labeled, unlabeled data 학습
    print("training unlabeled data...")
    infer_model, val_loss = train(model=model, optimizer=optimizer, epochs=1, train_loader=unlabeled_loader, val_loader=valid_loader, scheduler=scheduler, device=device)
    if best_val_loss < val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'model.pth')


training labeled data...


  0%|          | 0/3050 [00:00<?, ?it/s]

train_loss: 0.09522

  0%|          | 0/763 [00:00<?, ?it/s]

0.4806498363711731
Epoch [1], Train Loss : [0.20631] Val Loss : [0.48065]
detecting outliers...


  0%|          | 0/36600 [00:00<?, ?it/s]